In [154]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import cross_validate
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
import random as rd
from numpy.random import default_rng
import matplotlib.pyplot as plt
import time 

# Neuronka 2+1

In [155]:
n_feat = 2
n_neur = 3
c = 0.5
u = 0.1
b = 1

In [156]:
def init_wei(n_feat,n_neur):
    wei = default_rng().random((n_feat+1,n_neur))
    return wei

def learned_wei(fitted_wei):
    return fitted_wei

def ACT_fx(x_arr,wei,b,row,neur):
    ACT_res = (wei[neur][-1]*b) + np.sum(wei[neur][0:n_feat]*x_arr[row])
    return ACT_res

def ACT_fx_2(TF_res_in,wei,b,neur):
    ACT_res = (wei[neur][-1]*b) + np.sum(wei[neur][0:n_feat]*TF_res_in)
    return ACT_res

def SIG_fx_in(row,neur,wei,x_arr):
    TF_a = 1/(1+np.power(np.e,-ACT_fx(x_arr,wei,b,row,neur)))
    return TF_a

def SIG_fx_out(neur,TF_res_in,wei,x_arr):
    TF_a = 1/(1+np.power(np.e,-ACT_fx_2(TF_res_in,wei,b,neur)))
    return TF_a

def DELTA_Fx_outside(y_arr,TF_a,row):
    delta_out = (y_arr[row]-TF_a)*TF_a*(1-TF_a)
    return delta_out

def DELTA_Fx_inside(TF_a,wei,delta_out,neur):
    delta_in = TF_a*(1-TF_a)*(wei[neur][0:n_feat]*delta_out)
    return delta_in

def WEI_ch(delta,c,u,TF_a,wei_diff):
    wei_diff = (c*delta*TF_a)+(u*wei_diff)
    return wei_diff

def WEI_ch_inside(delta,c,u,x_arr,row,wei_diff):
    l = np.insert(x_arr[row],2,1)
    wei_diff = (c*delta*l)+(u*wei_diff)
    return wei_diff

def neuron_web(N_ep,x_arr,y_arr,choice,fitted_wei):
    TF_res_in = np.empty(0)
    delta_in = np.empty(0)
    wei_add_out = np.zeros(n_feat+1)
    wei_add_in_1 = np.zeros(n_feat+1)
    wei_add_in_2 = np.zeros(n_feat+1)
    if choice == 0:
        wei = init_wei(n_feat,n_neur)
    else:
        wei = learned_wei(fitted_wei)
    err_rate_res = [ ]
    err_rate = 0
    for epoch in range(N_ep):
        x_arr,y_arr = shuffle(x_arr,y_arr)
        err_rate_res.append(err_rate)
        err_rate = 0
        res = [ ]
        for row in range(len(x_arr)):
            for neur in range(n_neur-1):
                TF_res_in = np.append(TF_res_in,SIG_fx_in(row,neur,wei,x_arr))
            TF_res_out = SIG_fx_out(n_neur-1,TF_res_in,wei,x_arr)
            TF_res_in = np.insert(TF_res_in,2,1)
            delta_out = DELTA_Fx_outside(y_arr,TF_res_out,row)
            wei_add_out = WEI_ch(delta_out,c,u,TF_res_in,wei_add_out)
            TF_res_in = np.delete(TF_res_in,-1)
            delta_in = np.append(delta_in,DELTA_Fx_inside(TF_res_in,wei,delta_out,2)) 
            wei_add_in_1 = WEI_ch_inside(delta_in[1],c,u,x_arr,row,wei_add_in_1) 
            wei_add_in_2 = WEI_ch_inside(delta_in[0],c,u,x_arr,row,wei_add_in_2) 
            com_wei_add = np.vstack([wei_add_in_2,wei_add_in_1,wei_add_out])
            wei = wei+com_wei_add
            TF_res_in = np.empty(0)
            delta_in = np.empty(0)
            res.append(TF_res_out)
            err_rate += np.power(TF_res_out-y_arr[row],2)
    
    
    res = np.round(np.array(res))        
    prec = abs((abs(np.sum(res) - np.sum(y_arr))/len(y_arr))-1)

    return prec, wei

# Preprocessing

In [157]:
name_ls = [ ]
for i in range(0,58):
    name_ls.append(f'col{i}')

In [158]:
df = pd.read_csv('spambasedata.txt',names=name_ls)

In [159]:
df.head()

,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,...,col48,col49,col50,col51,col52,col53,col54,col55,col56,col57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


Normalizace

In [160]:
min_max_scaler = preprocessing.MinMaxScaler()
arr_df = min_max_scaler.fit_transform(df)
targets = arr_df[:,-1]

In [161]:
DF_corr = pd.DataFrame(arr_df)
DF_corr.columns = name_ls

Pearson

In [162]:
corr_pear = abs(DF_corr.corr(method='pearson')['col57'])
corr_pear_r = corr_pear.to_frame()
corr_pear_r.sort_values(by=['col57'],ascending = False).head(5)

,col57
col57,1.000000
col20,0.383234
col22,0.334787
col6,0.332117
col52,0.323629


Spearman

In [163]:
corr_spear = abs(DF_corr.corr(method='spearman')['col57'])
corr_spear_r = corr_spear.to_frame()
corr_spear_r.sort_values(by=['col57'],ascending = False).head(5)

,col57
col57,1.000000
col51,0.597854
col52,0.565633
col6,0.518778
col55,0.515157


# Testing 

Metoda 1 (za pomocí corr)

In [164]:
x_arr_m1 = np.vstack((arr_df[:,51],arr_df[:,52]))
x_arr_m1 = x_arr_m1.T
y_arr_m1 = targets
np.shape(x_arr_m1)

(4601, 2)

In [165]:
def cross_valid(data_x,data_y,n_div):
    div_data_x,div_data_y, res = [ ],[ ],[ ]
    data_x,data_y = shuffle(data_x,data_y)
    for div in range(n_div):
        div_data_x.append(data_x[(div*len(data_x))//n_div:((div+1)*len(data_x))//n_div][:])
        div_data_y.append(data_y[(div*len(data_y))//n_div:((div+1)*len(data_y))//n_div])
    for it in range(n_div):
        train_x, train_y, test_x, test_y = np.empty(2), np.empty(0), np.empty(2), np.empty(0)
        test_x, test_y = div_data_x[it], div_data_y[it]
        for i in range(n_div):
            if i == it:
                continue
            else:
                train_x = np.vstack((train_x,div_data_x[i]))
                train_y = np.append(train_y,div_data_y[i])
                
        train_x = np.delete(train_x,0,0)
        kk, fitted_wei = neuron_web(10,train_x,train_y,0,None)
        prec, wei  = neuron_web(1,test_x,test_y,1,fitted_wei)
        res.append(prec)
    return res

In [166]:
start_time = time.time()
res  = cross_valid(x_arr_m1,y_arr_m1,10)
print('%s sec'%(time.time()-start_time))

43.17350745201111 sec


In [167]:
name_ls = [ ]
for i in range(0,10):
    name_ls.append(f'fold{i+1}')
columns_lab = name_ls
columns_lab.append('Mean')
results = res
results.append(np.mean(res))

In [168]:
DF = pd.DataFrame(results)
DF = DF.T
DF.columns = columns_lab

In [169]:
DF

,fold1,fold2,fold3,fold4,fold5,fold6,fold7,fold8,fold9,fold10,Mean
0,0.873913,0.904348,0.869565,0.865217,0.891304,0.876087,0.908696,0.897826,0.9,0.887202,0.887416


Metoda 2 PCA

In [170]:
pca_x = arr_df.copy()
pca_x = np.delete(pca_x,-1,-1)
pca_y = targets

In [171]:
pca = PCA(n_components=2)
pca.fit(pca_x)
pca_X = pca.transform(pca_x)

In [172]:
np.shape(pca_X)

(4601, 2)

In [173]:
start_time = time.time()
res_PCA  = cross_valid(pca_X,pca_y,10)
print('%s sec'%(time.time()-start_time))

43.126375675201416 sec


In [174]:
name_ls = [ ]
for i in range(0,10):
    name_ls.append(f'fold{i+1}')
columns_lab = name_ls
columns_lab.append('Mean')
results = res_PCA
results.append(np.mean(res_PCA))

In [175]:
DF_PCA = pd.DataFrame(results)
DF_PCA = DF_PCA.T
DF_PCA.columns = columns_lab

In [176]:
DF_PCA

,fold1,fold2,fold3,fold4,fold5,fold6,fold7,fold8,fold9,fold10,Mean
0,0.986957,0.952174,0.993478,0.954348,0.984783,0.96087,0.965217,0.982609,0.976087,0.976139,0.973266
